In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [16]:
data = pd.read_csv('domain.csv')
# data = data.head(5)
data

,Domain
0,couponannie.com
1,thoughtshift.co.uk
2,jsfest.com.ua
3,zplux.com
4,sguru.org


In [17]:
email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b')
domain_pattern = re.compile(
"^(http[s]?:\\/\\/(www\\.)?|ftp:\\/\\/(www\\.)?|www\\.){1}([0-9A-Za-z-\\.@:%_\+~#=]+)+((\\.[a-zA-Z]{2,3})+)(/(.)*)?(\\?(.)*)?",
re.IGNORECASE,
)
image_pattern = re.compile(
"[^\\s]+(.*?)\\.(jpg|jpeg|png|gif|JPG|JPEG|PNG|GIF)$", re.IGNORECASE
)

In [19]:
def scrape_email(domain):
    list_emails = []
    try:
        if domain_pattern.match(domain):
            response = requests.get(domain)
        else:
            response = requests.get("http://" + domain + "/")
        
        
        page_source = response.text
        soup = BeautifulSoup(page_source, 'html.parser')
        emails = re.findall(email_pattern, str(soup))
            
        for email in emails:
            if email not in list_emails and image_pattern.match(email) is None:
                list_emails.append(email)
            
        if len(list_emails) == 0:
            print(f"No email found in {domain}")
        else:
            print(f"Email found in Domain {domain} : Email: {list_emails}")
            
        return list_emails
    except Exception as e:
        print(f"Error fetching emails for {domain}: {str(e)}")

In [20]:
data['Email'] = data['Domain'].apply(scrape_email)

No email found in couponannie.com
Email found in Domain thoughtshift.co.uk : Email: ['hello@thoughtshift.co.uk']
Email found in Domain jsfest.com.ua : Email: ['info@jsfest.com.ua', 'finance@jsfest.com.ua']
No email found in zplux.com
No email found in sguru.org


In [21]:
data

,Domain,Email
0,couponannie.com,[]
1,thoughtshift.co.uk,[hello@thoughtshift.co.uk]
2,jsfest.com.ua,"[info@jsfest.com.ua, finance@jsfest.com.ua]"
3,zplux.com,[]
4,sguru.org,[]


In [15]:
filename = 'domain_email.csv'
data.to_csv(filename,index = False)